# Curacao Risk Level for Extreme Sea Level Hazard Using Gornitz et al. (1994)
# Curacao Multi-Criteria Risk Level for Extreme Sea Level Hazard Using Mclaughlin and Cooper (2010)


In [221]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
from scipy.spatial import KDTree
import numpy as np

# Additional imports
import pystac_client
import xarray as xr
import pathlib
import os
import geopandas as gpd
import matplotlib as mpl
from matplotlib import colors
import rasterio

import cartopy.crs as ccrs
import cartopy
import plotly.graph_objects as go
import plotly.express as px
from rasterio.plot import show

import matplotlib.colors as mcolors
from shapely.geometry import Point
import pandas as pd

from tqdm import tqdm

import webbrowser
from pathlib import Path
import os
import time

from itertools import compress
import pandas as pd

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

from tqdm import tqdm

from shapely.geometry import box



In [222]:
Hazard_df=pd.read_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data_CVI_components\Hazard\Present\Curacao_Hazards_classified.csv")
Hazard_df

,distance,sampled points,ESL_RCP0.0_Time_Horizon2000,Sig. WaveHeight 95th percentile,Shoreline change_rate,Land_Subsidence,geometry,Extreme Sea Level - PC - Hazard level,Sig. WaveHeight 95th% - PC - Hazard level,Shoreline change rate - PC - Hazard level
0,0,0.0,0.869869,1.695959,NaN,NaN,POINT (-68.8483397 12.061695899999998),2.0,1.0,NaN
1,100,0.0,0.869869,1.695959,NaN,NaN,POINT (-68.84911617380439 12.06224646915572),2.0,1.0,NaN
2,200,0.0,0.869869,1.695959,NaN,NaN,POINT (-68.84990212744901 12.062863188234783),2.0,1.0,NaN
3,300,0.0,0.869869,1.695959,NaN,NaN,POINT (-68.85069884613627 12.063463122779966),2.0,1.0,NaN
4,400,0.0,0.869869,1.695959,NaN,NaN,POINT (-68.85135637829056 12.064185009066897),2.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...
2435,243500,0.0,0.869869,1.695959,-0.218747,NaN,POINT (-68.84468462007678 12.060052302736732),2.0,1.0,3.0
2436,243600,0.0,0.869869,1.695959,-0.218747,NaN,POINT (-68.84558616670871 12.060483706569734),2.0,1.0,3.0
2437,243700,0.0,0.869869,1.695959,-0.218747,NaN,POINT (-68.84652050289856 12.060837534484437),2.0,1.0,3.0
2438,243800,0.0,0.869869,1.695959,-0.218747,NaN,POINT (-68.84743760359714 12.061236116783006),2.0,1.0,3.0


In [223]:
Exposure_df=pd.read_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data_CVI_components\Exposure\Curacao_Exposure_classified.csv")
Exposure_df

,distance,sampled points,slope,pop_count,Delta_DEM--LECZ_area_per_LECZ_mean_elev,built_up_perc,Sediment Type_Exposure_level,geometry,Nearshore Slope - Exposure level,Population - Exposure level,Elev (DeltaDEM) - Exposure level,Built-up % - Exposure level,Sediment_Type - Exposure level
0,0,4.0,4.010837,140.0,289736.126650,17.732094,4.0,POINT (-68.8483397 12.061695899999998),3.0,1.0,2.0,2.0,4.0
1,100,4.0,4.010837,140.0,289736.126650,17.732094,4.0,POINT (-68.84911617380439 12.06224646915572),3.0,1.0,2.0,2.0,4.0
2,200,4.0,4.010837,140.0,289736.126650,17.732094,4.0,POINT (-68.84990212744901 12.062863188234783),3.0,1.0,2.0,2.0,4.0
3,300,4.0,4.010837,140.0,289736.126650,17.732094,NaN,POINT (-68.85069884613627 12.063463122779966),3.0,1.0,2.0,2.0,NaN
4,400,4.0,0.003349,136.0,356249.967335,17.506038,NaN,POINT (-68.85135637829056 12.064185009066897),5.0,1.0,3.0,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2435,243500,4.0,4.010837,74.0,244939.371945,15.372980,4.0,POINT (-68.84468462007678 12.060052302736732),3.0,1.0,2.0,2.0,4.0
2436,243600,4.0,4.010837,74.0,244939.371945,15.372980,4.0,POINT (-68.84558616670871 12.060483706569734),3.0,1.0,2.0,2.0,4.0
2437,243700,4.0,4.010837,140.0,289736.126650,17.732094,4.0,POINT (-68.84652050289856 12.060837534484437),3.0,1.0,2.0,2.0,4.0
2438,243800,4.0,4.010837,140.0,289736.126650,17.732094,4.0,POINT (-68.84743760359714 12.061236116783006),3.0,1.0,2.0,2.0,4.0


Curacao Risk Level for Extreme Sea Level Hazard Using Gornitz et al. (1994) CVI6 

In [224]:
# UNCOMMENT AS NEEDED

# gornitz_hazard_used="Extreme Sea Level - PC - Hazard level"
# gornitz_hazard_used="Sig. WaveHeight 95th% - PC - Hazard level"
gornitz_hazard_used="Shoreline change rate - PC - Hazard level"



title_name=gornitz_hazard_used.replace(" - Hazard level","")

gornitz_ESL=pd.concat([Hazard_df.iloc[:,0],Hazard_df[gornitz_hazard_used],Exposure_df.iloc[:,-5:]],axis=1)
gornitz_ESL

,distance,Shoreline change rate - PC - Hazard level,Nearshore Slope - Exposure level,Population - Exposure level,Elev (DeltaDEM) - Exposure level,Built-up % - Exposure level,Sediment_Type - Exposure level
0,0,NaN,3.0,1.0,2.0,2.0,4.0
1,100,NaN,3.0,1.0,2.0,2.0,4.0
2,200,NaN,3.0,1.0,2.0,2.0,4.0
3,300,NaN,3.0,1.0,2.0,2.0,NaN
4,400,NaN,5.0,1.0,3.0,2.0,NaN
...,...,...,...,...,...,...,...
2435,243500,3.0,3.0,1.0,2.0,2.0,4.0
2436,243600,3.0,3.0,1.0,2.0,2.0,4.0
2437,243700,3.0,3.0,1.0,2.0,2.0,4.0
2438,243800,3.0,3.0,1.0,2.0,2.0,4.0


In [225]:
indicator_num=len(gornitz_ESL.columns)-1
indicator_num

6

In [226]:
gornitz_ESL_filled=gornitz_ESL.copy()
gornitz_ESL_filled["Risk Level"]=(gornitz_ESL[gornitz_hazard_used]+(np.nansum(gornitz_ESL.iloc[:,-5:],axis=1)/len(gornitz_ESL.iloc[0,-5:])))/2
gornitz_ESL_filled

,distance,Shoreline change rate - PC - Hazard level,Nearshore Slope - Exposure level,Population - Exposure level,Elev (DeltaDEM) - Exposure level,Built-up % - Exposure level,Sediment_Type - Exposure level,Risk Level
0,0,NaN,3.0,1.0,2.0,2.0,4.0,NaN
1,100,NaN,3.0,1.0,2.0,2.0,4.0,NaN
2,200,NaN,3.0,1.0,2.0,2.0,4.0,NaN
3,300,NaN,3.0,1.0,2.0,2.0,NaN,NaN
4,400,NaN,5.0,1.0,3.0,2.0,NaN,NaN
...,...,...,...,...,...,...,...,...
2435,243500,3.0,3.0,1.0,2.0,2.0,4.0,2.7
2436,243600,3.0,3.0,1.0,2.0,2.0,4.0,2.7
2437,243700,3.0,3.0,1.0,2.0,2.0,4.0,2.7
2438,243800,3.0,3.0,1.0,2.0,2.0,4.0,2.7


In [227]:


df = gornitz_ESL.iloc[:,1:]

# Identify columns with NaN values
nan_columns = df.columns[df.isna().any()].tolist()

# Create a new column 'NaN_Columns' containing list of column names with NaN values for each row
gornitz_ESL_filled['Uncertainty_Columns'] = df.apply(lambda row: [col for col in nan_columns if pd.isna(row[col])], axis=1)
gornitz_ESL_filled['Uncertainty_Columns_length'] = gornitz_ESL_filled['Uncertainty_Columns'].apply(len)

df


,Shoreline change rate - PC - Hazard level,Nearshore Slope - Exposure level,Population - Exposure level,Elev (DeltaDEM) - Exposure level,Built-up % - Exposure level,Sediment_Type - Exposure level
0,NaN,3.0,1.0,2.0,2.0,4.0
1,NaN,3.0,1.0,2.0,2.0,4.0
2,NaN,3.0,1.0,2.0,2.0,4.0
3,NaN,3.0,1.0,2.0,2.0,NaN
4,NaN,5.0,1.0,3.0,2.0,NaN
...,...,...,...,...,...,...
2435,3.0,3.0,1.0,2.0,2.0,4.0
2436,3.0,3.0,1.0,2.0,2.0,4.0
2437,3.0,3.0,1.0,2.0,2.0,4.0
2438,3.0,3.0,1.0,2.0,2.0,4.0


In [228]:

gornitz_ESL_filled["Confindence %"]=100*(indicator_num-gornitz_ESL_filled['Uncertainty_Columns'].apply(len))/indicator_num
gornitz_ESL_filled

,distance,Shoreline change rate - PC - Hazard level,Nearshore Slope - Exposure level,Population - Exposure level,Elev (DeltaDEM) - Exposure level,Built-up % - Exposure level,Sediment_Type - Exposure level,Risk Level,Uncertainty_Columns,Uncertainty_Columns_length,Confindence %
0,0,NaN,3.0,1.0,2.0,2.0,4.0,NaN,[Shoreline change rate - PC - Hazard level],1,83.333333
1,100,NaN,3.0,1.0,2.0,2.0,4.0,NaN,[Shoreline change rate - PC - Hazard level],1,83.333333
2,200,NaN,3.0,1.0,2.0,2.0,4.0,NaN,[Shoreline change rate - PC - Hazard level],1,83.333333
3,300,NaN,3.0,1.0,2.0,2.0,NaN,NaN,"[Shoreline change rate - PC - Hazard level, Se...",2,66.666667
4,400,NaN,5.0,1.0,3.0,2.0,NaN,NaN,"[Shoreline change rate - PC - Hazard level, Se...",2,66.666667
...,...,...,...,...,...,...,...,...,...,...,...
2435,243500,3.0,3.0,1.0,2.0,2.0,4.0,2.7,[],0,100.000000
2436,243600,3.0,3.0,1.0,2.0,2.0,4.0,2.7,[],0,100.000000
2437,243700,3.0,3.0,1.0,2.0,2.0,4.0,2.7,[],0,100.000000
2438,243800,3.0,3.0,1.0,2.0,2.0,4.0,2.7,[],0,100.000000


In [229]:
# Read the CSV file into a DataFrame
circle_center_df = Exposure_df


# Identify the correct geometry column (replace 'geometry_column_name' with the actual column name)
geometry_column_name = 'geometry'
circle_center_df['x'] = circle_center_df[geometry_column_name].apply(lambda point_str: float(point_str.strip('POINT ()').split()[0]))
circle_center_df['y'] = circle_center_df[geometry_column_name].apply(lambda point_str: float(point_str.strip('POINT ()').split()[1]))

circle_center_df = circle_center_df.drop(columns=[geometry_column_name])

# Create the GeoDataFrame with the correct geometry column
gornitz_ESL_filled_gdf = gpd.GeoDataFrame(gornitz_ESL_filled, geometry=gpd.points_from_xy(circle_center_df.x, circle_center_df.y))



gornitz_ESL_filled_gdf

,distance,Shoreline change rate - PC - Hazard level,Nearshore Slope - Exposure level,Population - Exposure level,Elev (DeltaDEM) - Exposure level,Built-up % - Exposure level,Sediment_Type - Exposure level,Risk Level,Uncertainty_Columns,Uncertainty_Columns_length,Confindence %,geometry
0,0,NaN,3.0,1.0,2.0,2.0,4.0,NaN,[Shoreline change rate - PC - Hazard level],1,83.333333,POINT (-68.84834 12.06170)
1,100,NaN,3.0,1.0,2.0,2.0,4.0,NaN,[Shoreline change rate - PC - Hazard level],1,83.333333,POINT (-68.84912 12.06225)
2,200,NaN,3.0,1.0,2.0,2.0,4.0,NaN,[Shoreline change rate - PC - Hazard level],1,83.333333,POINT (-68.84990 12.06286)
3,300,NaN,3.0,1.0,2.0,2.0,NaN,NaN,"[Shoreline change rate - PC - Hazard level, Se...",2,66.666667,POINT (-68.85070 12.06346)
4,400,NaN,5.0,1.0,3.0,2.0,NaN,NaN,"[Shoreline change rate - PC - Hazard level, Se...",2,66.666667,POINT (-68.85136 12.06419)
...,...,...,...,...,...,...,...,...,...,...,...,...
2435,243500,3.0,3.0,1.0,2.0,2.0,4.0,2.7,[],0,100.000000,POINT (-68.84468 12.06005)
2436,243600,3.0,3.0,1.0,2.0,2.0,4.0,2.7,[],0,100.000000,POINT (-68.84559 12.06048)
2437,243700,3.0,3.0,1.0,2.0,2.0,4.0,2.7,[],0,100.000000,POINT (-68.84652 12.06084)
2438,243800,3.0,3.0,1.0,2.0,2.0,4.0,2.7,[],0,100.000000,POINT (-68.84744 12.06124)


In [230]:
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv(r'C:\Users\rowe\Documents\GitHub\coclicodata\.env')

# Access the Mapbox Access Token
mapbox_access_token = os.getenv('MAPBOX_ACCESS_TOKEN')

print("Mapbox Access Token:", mapbox_access_token)

Mapbox Access Token: sk.eyJ1IjoiZ2xvYmFsLWRhdGEtdmlld2VyIiwiYSI6ImNsMnB0dmlscTFrZnEzY211cWxna3Bxb3AifQ.OQ3P9PEjZUiErvjrwVbsag


In [231]:
color_scale = ['#33f510','#3CB371','#FFFF00', '#FFA500','#FF4500']


fig2 = go.Figure(go.Scattermapbox(
    lat=gornitz_ESL_filled_gdf["geometry"].y,
    lon=gornitz_ESL_filled_gdf["geometry"].x,
    mode='markers',
    marker=dict(
        size=10,
        color=gornitz_ESL_filled_gdf["Risk Level"],  # Set color to red

        colorscale=color_scale,  # Specify the discrete color scale
        cmin=1,  # Set the minimum value for the color scale
        cmax=5,  # Set the maximum value for the color scale
        colorbar=dict(
            # title=""  # Set color bar title?
            title = "Risk Level", 
            tickvals=[1, 2, 3, 4, 5],  # Set the tick values
            ticktext=['1', '2', '3', '4', '5'],  # Set the tick labels    
        )
    ),
    hoverinfo='text',
    hovertext=gornitz_ESL_filled_gdf["Uncertainty_Columns"],
    showlegend=False
))

# Update layout properties
fig2.update_layout(
    title = "Curacao Risk Level for " +title_name+" Using Gornitz et al. (1994) CVI6 ",
    mapbox_style="carto-positron",
    mapbox=dict(
        zoom=9.5,
        center=dict(lat=12.18625, lon=-68.95)
    ),		

    margin=dict(l=0, r=50, b=0, t=100),
    legend=dict(
    x=1.15,
    y=1.0,
    traceorder="normal",
    orientation="v"
),
    width=800,  # Set width of the plot
    height=600  # Set height of the plot
)

# Show the plot
fig2.show()

directory_path = r"C:\Users\rowe\Documents\GitHub\global-coastal-atlas\STAC\visualization\SOTC_transect_mapping_visual\Plots\Poster\FINAL-CVI"
pio.write_image(fig2, os.path.join(directory_path,gornitz_ESL_filled_gdf.columns[1].replace('Hazard', 'Risk') +"- Curacao.png"))

In [232]:
# import plotly.graph_objects as go

# color_scale = ['#33f510', '#3CB371', '#FFFF00', '#FFA500', '#FF4500']

# fig2 = go.Figure(go.Scattermapbox(
#     lat=gornitz_ESL_filled_gdf["geometry"].y,
#     lon=gornitz_ESL_filled_gdf["geometry"].x,
#     mode='markers',
#     marker=dict(
#         size=15,
#         color=gornitz_ESL_filled_gdf["Risk Level"],  # Set color to red
#         colorscale=color_scale,  # Specify the discrete color scale
#         cmin=1,  # Set the minimum value for the color scale
#         cmax=5,  # Set the maximum value for the color scale
#         colorbar=dict(
#             title="Risk Level",
#             tickvals=[1, 2, 3, 4, 5],  # Set the tick values
#             ticktext=['1', '2', '3', '4', '5'],  # Set the tick labels
#         )
#     ),
#     hoverinfo='text',
#     hovertext=gornitz_ESL_filled_gdf["Uncertainty_Columns"],
#     showlegend=False
# ))

# # Update layout properties
# fig2.update_layout(
#     title="Curacao Risk Level for Extreme Sea Level Hazard Using Gornitz et al. (1994) CVI6",
#     mapbox=dict(
#         accesstoken=mapbox_access_token,  # Set the Mapbox access token here
#         style="satellite-streets",
#         zoom=5,
#         center=dict(lat=12.18625, lon=-68.95)
#     ),
#     margin=dict(l=0, r=50, b=0, t=100),
#     legend=dict(
#         x=1.15,
#         y=1.0,
#         traceorder="normal",
#         orientation="v"
#     ),
#     width=800,  # Set width of the plot
#     height=600  # Set height of the plot
# )

# # Show the plot
# fig2.show()


Curacao Multi-Criteria Risk Level for Extreme Sea Level Hazard Using Mclaughlin and Cooper (2010)